# Course Project No 3 - Part 2 - House prices - 2 of 2

## Import base libraries

Let's import all the base libraries that we will use in this Notebook

In [40]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline


## Load the datasets

In this Notebook, we will load the cleaned and adapted version of our two datasets. See introduction in the Notebook *house-prices-solution-1-of-2.ipynb* to know form where those files arise.


In [41]:
train_df = pd.read_csv(os.path.join('data','house-prices-cleaned.csv'))
print("Size of the train dataset:", train_df.shape)
train_df.head(3)

Size of the train dataset: (2207, 254)


,Lot Frontage,Lot Area,Lot Shape,Land Slope,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,...,House New,Extra Flr SF,Extra Flr SF Exists,2nd Flr SF Exists,Low Qual Fin SF Exists,3Ssn Porch Exists,Misc Val Exists,Enclosed Porch Exists,Screen Porch Exists,Continuous Sum
0,68.0,8795,3,3,7,5,10,10,0.0,4,...,0,0,0,1,0,0,0,0,0,346.0
1,75.0,10170,4,3,6,6,59,59,522.0,3,...,0,0,0,0,0,0,0,0,0,597.0
2,21.0,2001,4,3,4,5,40,40,80.0,3,...,0,0,0,1,0,0,0,0,0,101.0


In [42]:
test_df = pd.read_csv(os.path.join('data','house-prices-test-cleaned.csv'))
print("Size of the test dataset:", test_df.shape)
test_df.head(3)

Size of the test dataset: (500, 254)


,Lot Frontage,Lot Area,Lot Shape,Land Slope,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,...,House New,Extra Flr SF,Extra Flr SF Exists,2nd Flr SF Exists,Low Qual Fin SF Exists,3Ssn Porch Exists,Misc Val Exists,Enclosed Porch Exists,Screen Porch Exists,Continuous Sum
0,68.0,11275,3,2,6,7,78,60,480.0,3,...,0,0,0,1,0,0,0,1,0,548.0
1,65.0,9757,4,2,5,7,16,16,0.0,3,...,0,0,0,0,0,0,0,0,1,131.0
2,86.0,11065,3,3,8,5,4,4,788.0,4,...,1,0,0,1,0,0,0,0,0,1125.0


Ok, so here we are, cleaned Train and adapted Test datasets ready for machine learning work.

## Model Fitting

### Create feature matrix and vector result

In [43]:
# Build feature matrix X and result vector y narray from the train and test dataframe
X_tr = train_df.drop('SalePrice', axis=1).values
y_tr = train_df['SalePrice'].values
X_te = test_df.drop('SalePrice', axis=1).values
y_te = test_df['SalePrice'].values

X = np.concatenate((X_tr, X_te))
y = np.concatenate((y_tr, y_te))

print("Shape of X_tr variable:",X_tr.shape)
print("Shape of y_tr variable:",y_tr.shape)

print("Shape of X_te variable:",X_te.shape)
print("Shape of y_te variable:",y_te.shape)

print("Shape of X variable:",X.shape)
print("Shape of y variable:",y.shape)

Shape of X_tr variable: (2207, 253)
Shape of y_tr variable: (2207,)
Shape of X_te variable: (500, 253)
Shape of y_te variable: (500,)
Shape of X variable: (2707, 253)
Shape of y variable: (2707,)


In [44]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selectK = SelectKBest(chi2, k=20)
X_new = selectK.fit_transform(X_tr, y_tr)


In [45]:
mask = selectK.get_support() #list of booleans
new_features = [] # The list of your K best features

for bool, feature in zip(mask, train_df.drop(['SalePrice'],axis=1).columns):
    if bool:
        new_features.append(feature)

print(new_features)

['Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'Extra Flr SF', 'Continuous Sum']
